In [1]:
import pandas as pd
import numpy as np
import acquire

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

### 1. Attendance Data

Load the attendance.csv file and calculate an attendnace percentage for each student.   
One half day is worth 50% of a full day, and 10 tardies is equal to one absence.  

You should end up with something like this:    
name   
Billy     0.5250    
Jane      0.6875  
John      0.9125  
Sally     0.7625  
Name: grade, dtype: float64

In [ ]:
# load the dataset
attend_df = pd.read_csv('attendance.csv')
attend_df

In [ ]:
# try a melt
# df = df.melt(id_vars=['student'], var_name='subject', value_name='grade')
attend_df = attend_df.melt(id_vars=['Unnamed: 0'])
attend_df

In [ ]:
# rename columns
attend_df = attend_df.rename(columns={'Unnamed: 0': 'student', 'variable': 'date', 'value': 'code'})
attend_df.head()

In [ ]:
# assign number value to letter codes, P=present=1, A=absent=0, H=half_day=.5, T=tardy=-.1
# create column for each code
attend_df['present'] = attend_df[attend_df.code == 'P'].code
attend_df['absent'] = attend_df[attend_df.code == 'A'].code
attend_df['half_day'] = attend_df[attend_df.code == 'H'].code
attend_df['tardy'] = attend_df[attend_df.code == 'T'].code
attend_df.head()

In [ ]:
# fill null results with 0 value
attend_df = attend_df.fillna(0)

In [ ]:
# replace letter with numeric value
attend_df['present'] = attend_df['present'].replace('P', 1)
attend_df['absent'] = attend_df['absent'].replace('A', 0)
attend_df['half_day'] = attend_df['half_day'].replace('H', .5)
attend_df['tardy'] = attend_df['tardy'].replace('T', .9)

attend_df.head()

In [ ]:
# don't need code or abesent columns (if not present, half_day, or tardy then absent)
# drop code and absent columns
attend_df = attend_df.drop(columns=['code', 'absent'])
attend_df

In [ ]:
# add total column?
attend_df['average'] = attend_df.present + attend_df.half_day + attend_df.tardy
attend_df.head()

In [ ]:
# df.groupby('student').mean()
attend_df.groupby('student').mean()
# this is close, has correct values for points according to table above

In [ ]:
# try dropping all but student and average
attend_df = attend_df.drop(columns=['present', 'half_day', 'tardy'])
attend_df.groupby('student').mean()

### 2. Coffee Levels

Read the coffee_levels.csv file.  
Transform the data so that each carafe is in it's own column.  
Is this the best shape for the data?

In [ ]:
coffee = pd.read_csv('coffee_levels.csv')
coffee.head()

In [ ]:
# try a pivot table
# df = df.pivot_table('value', ['country', 'year'], 'measure')
# students.pivot(index='date', columns='var')
coffee = coffee.pivot(index='hour', columns='coffee_carafe')


In [ ]:
coffee.columns = ['x_amount', 'y_amount', 'z_amount']
coffee

In [ ]:
coffee.shape

### 3. Cake Recipes

Read the cake_recipes.csv data.   
This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.  
Tidy the data as necessary.  
Which recipe, on average, is the best? recipe b  
Which oven temperature, on average, produces the best results? 275  
Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees


In [2]:
cake = pd.read_csv('cake_recipes.csv')
cake

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [3]:
# treatments.melt(id_vars=['name'], var_name='treatment', value_name='response')
cake = cake.melt(id_vars=['recipe:position'], var_name='temp', value_name='score')
cake.head()

,recipe:position,temp,score
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207


In [4]:
# split recipe and position into separate columns
rec_pos = cake["recipe:position"].str.split(':', expand=True)
rec_pos.columns = ['recipe', 'rack_position']
cake = pd.concat([rec_pos, cake], axis=1)
cake.head()

,recipe,rack_position,recipe:position,temp,score
0,a,bottom,a:bottom,225,61.738655
1,a,top,a:top,225,51.709751
2,b,bottom,b:bottom,225,57.095320
3,b,top,b:top,225,82.455004
4,c,bottom,c:bottom,225,96.470207


In [5]:
# drop recipe:position column
cake = cake.drop(columns='recipe:position')
cake.head()

,recipe,rack_position,temp,score
0,a,bottom,225,61.738655
1,a,top,225,51.709751
2,b,bottom,225,57.095320
3,b,top,225,82.455004
4,c,bottom,225,96.470207


In [17]:
# Which recipe, on average, is the best? recipe b
cake.groupby('recipe')['score'].max()

recipe
a    98.786784
b    99.248541
c    96.470207
d    96.873178
Name: score, dtype: float64

In [29]:
# Which oven temperature, on average, produces the best results? 275
cake.groupby('temp')['score'].mean()

temp
225    71.306022
250    66.577437
275    74.886754
300    66.627655
Name: score, dtype: float64

In [28]:
# Which combination of recipe, rack position, and temperature gives the best result? 
# recipe b, bottom rack, 300 degrees
# df.sort_values(by=['col1', 'col2'])
cake.sort_values(by=['score'], ascending=False).head(1)

,recipe,rack_position,temp,score
26,b,bottom,300,99.248541
